## Step 1: The Foundation (Imports and Setup)
Every Python script starts with importing the necessary libraries and setting up the environment. This is like laying the foundation for a house.

In [1]:
import random
import csv
from datetime import datetime
from torch.utils.data import Dataset
import pandas as pd
import os
import numpy as np
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import sys

# -- Basic Setup --
# Set the device to use the GPU if available, otherwise use the CPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


### Define dataset loader

PyTorch uses a Dataset object to handle data loading. Since our model will take three different kinds of input (sensor data, image 1, and image 2), we need to create a special class that tells PyTorch how to retrieve one sample of each, along with its corresponding label.

This class will have three essential methods:

__init__: Initializes the dataset by storing our feature and label arrays.

__len__: Returns the total number of samples in the dataset.

__getitem__: Fetches a single data sample at a given index.

Here is the code for it. Add this to your script:

In [2]:
# Define dataset loader
class CustomDatasetRes(Dataset):
    def __init__(self, features1, features2, features3, labels):
        self.features1 = features1
        self.features2 = features2
        self.features3 = features3
        self.labels = labels

    def __len__(self):
        return len(self.features1)
    
    def __getitem__(self, index):
        return self.features1[index], self.features2[index], self.features3[index], self.labels[index]

### Helper Functions
Next, we'll add a few helper functions. These functions will perform common tasks that we'll need later, like displaying results, scaling data, and ensuring our experiments are reproducible.

1. display_result

This function takes the true labels (y_test) and the model's predicted labels (y_pred) and prints out standard performance metrics like accuracy, precision, recall, and F1-score.

In [3]:
def display_result(y_test, y_pred):
    print('Accuracy score : ', accuracy_score(y_test, y_pred))
    print('Precision score : ', precision_score(y_test, y_pred, average='weighted'))
    print('Recall score : ', recall_score(y_test, y_pred, average='weighted'))
    print('F1 score : ', f1_score(y_test, y_pred, average='weighted'))

2. scaled_data

This function uses Scikit-learn's StandardScaler to normalize the sensor (CSV) data. Scaling is crucial because it ensures that features with larger value ranges don't dominate the learning process. Notice there are two functions with the same name in the original code. In Python, the last definition of a function is the one that gets used. We will add both for completeness, but just know that the first one is effectively overwritten by the second.

In [4]:
def scale_data(X_train, X_test, X_val):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    X_val_scaled = scaler.transform(X_val)
    return X_train_scaled, X_test_scaled, X_val_scaled

def scaled_data(X_train):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    return X_train_scaled

3. set_seed

This is a very important function for reproducibility. Machine learning involves a lot of randomness (e.g., initializing model weights, shuffling data). By setting a "seed," we ensure that the sequence of random numbers is the same every time we run the code, which means we'll get the exact same results.

In [5]:
def set_seed(seed=0):
    # Sets the environment variable for Python's hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    # Sets the seed for NumPy's random number generator
    np.random.seed(seed)
    # Sets the seed for Python's built-in random module
    random.seed(seed)
    # Sets the seed for PyTorch's random number generator
    torch.manual_seed(seed)
    # If using a GPU, sets the seed for all CUDA devices
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # For multi-GPU setups
    # Ensures deterministic behavior in cuDNN (CUDA Deep Neural Network library)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

### loading and preprocessing the data.

The function loadClientsData is designed for a federated learning scenario. It reads data from separate files for each participant (or "client"), cleans it, aligns the different data types (sensor vs. image), and splits it into training and testing sets for each client.

Because this function is quite long, we'll build it in a few parts.

#### Part 1: Initializing and Processing Training Data
First, we'll define the function, list the subject IDs we want to load, and create empty dictionaries to store each client's data. Then, we'll start a loop to process each subject one by one. Inside the loop, we'll begin by loading and cleaning the training data.

This involves:

Reading the sensor data from a CSV file.

Removing rows with missing values and any duplicate rows.

Dropping columns that we don't need (like the 'Infrared' sensor readings).

Loading the corresponding image, label, and timestamp data from .npy files.

#### Part 2: Aligning and Preparing Training Data
After loading the raw data, we face a common problem: the datasets don't perfectly match. Because we dropped rows with missing values from the sensor (CSV) data, there are now timestamps in our image data that no longer have a corresponding entry in the sensor data.

We need to align them by removing the image samples that don't have a matching sensor reading.

After alignment, we'll prepare the data for the model:

Set the seed for reproducibility.

Separate features from labels.

One-hot encode the labels, converting them into a format suitable for the model's output layer (e.g., class 3 becomes [0, 0, 0, 1, 0, ...]).

Scale the numeric sensor data and the image pixel values.

Reshape the images to the format expected by the convolutional layers.

#### Part 3: Processing the Test Data and Finalizing the Function
The logic here is identical to what we just did for the training data:

Load the test sensor data (_test.csv) and test image data (_test.npy).

Clean the sensor data by removing missing values and unnecessary columns.

Align the test image data with the cleaned test sensor data.

Prepare the aligned test data (one-hot encode labels, scale features, reshape images).

Store all the processed training and test arrays into our dictionaries.

Increment the clint_index and repeat the process for the next subject.

After the loop finishes, the function returns all the dictionaries containing the data for every client.

In [6]:
def loadClientsData():
    subs = [1, 3, 4, 7, 10, 11, 12, 13, 14, 15, 16, 17]
    X_train_csv_scaled_splits = {}
    X_test_csv_scaled_splits = {}
    Y_train_csv_splits = {}
    Y_test_csv_splits = {}
    X_train_1_scaled_splits = {}
    X_test_1_scaled_splits = {}
    Y_train_1_splits = {}
    Y_test_1_splits = {}
    X_train_2_scaled_splits = {}
    X_test_2_scaled_splits = {}
    Y_train_2_splits = {}
    Y_test_2_splits = {}
    clint_index = 0
    for sub_ in subs:
        # --- Load and clean TRAINING sensor data (CSV) ---
        SUB_train = pd.read_csv('./dataset/Sensor + Image/{}_sensor_train.csv'.format(sub_), skiprows=1)
        SUB_train.head()
        
        SUB_train.isnull().sum()
        NA_cols = SUB_train.columns[SUB_train.isnull().any()]
        SUB_train.dropna(inplace=True)
        SUB_train.drop_duplicates(inplace=True)
        
        times_train = SUB_train['Time']
        list_DROP = ['Infrared 1',
                     'Infrared 2',
                     'Infrared 3',
                     'Infrared 4',
                     'Infrared 5',
                     'Infrared 6']
        SUB_train.drop(list_DROP, axis=1, inplace=True)
        SUB_train.drop(NA_cols, axis=1, inplace=True)  # drop NAN COLS

        SUB_train.set_index('Time', inplace=True)
        SUB_train.head()

        # --- Load TRAINING image data from both cameras ---
        cam = '1'
        image_train = './dataset/Sensor + Image' + '/' + '{}_image_1_train.npy'.format(sub_)
        name_train = './dataset/Sensor + Image' + '/' + '{}_name_1_train.npy'.format(sub_)
        label_train = './dataset/Sensor + Image' + '/' + '{}_label_1_train.npy'.format(sub_)

        img_1_train = np.load(image_train)
        label_1_train = np.load(label_train)
        name_1_train = np.load(name_train)

        cam = '2'
        image_train = './dataset/Sensor + Image' + '/' + '{}_image_2_train.npy'.format(sub_)
        name_train = './dataset/Sensor + Image' + '/' + '{}_name_2_train.npy'.format(sub_)
        label_train = './dataset/Sensor + Image' + '/' + '{}_label_2_train.npy'.format(sub_)

        img_2_train = np.load(image_train)
        label_2_train = np.load(label_train)
        name_2_train = np.load(name_train)

        # --- Align the training data by removing samples not present in the cleaned CSV ---
        redundant_1 = list(set(name_1_train) - set(times_train))
        redundant_2 = list(set(name_2_train) - set(times_train))
        
        ind = np.arange(0, len(img_1_train))

        red_in1 = ind[np.isin(name_1_train, redundant_1)]
        name_1_train = np.delete(name_1_train, red_in1)
        img_1_train = np.delete(img_1_train, red_in1, axis=0)
        label_1_train = np.delete(label_1_train, red_in1)

        red_in2 = ind[np.isin(name_2_train, redundant_2)]
        name_2_train = np.delete(name_2_train, red_in2)
        img_2_train = np.delete(img_2_train, red_in2, axis=0)
        label_2_train = np.delete(label_2_train, red_in2)
        
        # --- Prepare the final aligned training data ---
        data_train = SUB_train.loc[name_1_train].values

        set_seed()
        X_csv_train, y_csv_train = data_train[:, :-1], data_train[:, -1]
        
        # Remap label 20 to 0 for consistency
        y_csv_train = np.where(y_csv_train == 20, 0, y_csv_train)
        label_1_train = np.where(label_1_train == 20, 0, label_1_train)
        label_2_train = np.where(label_2_train == 20, 0, label_2_train)

        # One-hot encode the labels for PyTorch
        Y_csv_train = torch.nn.functional.one_hot(torch.from_numpy(y_csv_train).long(), 12).float()
        Y_train_1 = torch.nn.functional.one_hot(torch.from_numpy(label_1_train).long(), 12).float()
        Y_train_2 = torch.nn.functional.one_hot(torch.from_numpy(label_2_train).long(), 12).float()

        # Scale the sensor data
        X_csv_train_scaled = scaled_data(X_csv_train)

        X_train_1 = img_1_train
        y_train_1 = label_1_train
        
        X_train_2 = img_2_train
        y_train_2 = label_2_train

        # Reshape images to (samples, height, width, channels)
        shape1, shape2 = 32, 32
        X_train_1 = X_train_1.reshape(X_train_1.shape[0], shape1, shape2, 1)
        X_train_2 = X_train_2.reshape(X_train_2.shape[0], shape1, shape2, 1)

        # Scale image pixel values to be between 0 and 1
        X_train_1_scaled = X_train_1 / 255.0
        X_train_2_scaled = X_train_2 / 255.0

        # --- Load and clean TEST sensor data (CSV) ---
        SUB_test = pd.read_csv('./dataset/Sensor + Image/{}_sensor_test.csv'.format(sub_), skiprows=1)
        SUB_test.head()
        
        SUB_test.isnull().sum()
        NA_cols = SUB_test.columns[SUB_test.isnull().any()]
        SUB_test.dropna(inplace=True)
        SUB_test.drop_duplicates(inplace=True)

        times_test = SUB_test['Time']
        SUB_test.drop(list_DROP, axis=1, inplace=True)
        SUB_test.drop(NA_cols, axis=1, inplace=True)

        SUB_test.set_index('Time', inplace=True)
        SUB_test.head()

        # --- Load TEST image data from both cameras ---
        image_test = './dataset/Sensor + Image' + '/' + '{}_image_1_test.npy'.format(sub_)
        name_test = './dataset/Sensor + Image' + '/' + '{}_name_1_test.npy'.format(sub_)
        label_test = './dataset/Sensor + Image' + '/' + '{}_label_1_test.npy'.format(sub_)
        img_1_test = np.load(image_test)
        label_1_test = np.load(label_test)
        name_1_test = np.load(name_test)

        image_test = './dataset/Sensor + Image' + '/' + '{}_image_2_test.npy'.format(sub_)
        name_test = './dataset/Sensor + Image' + '/' + '{}_name_2_test.npy'.format(sub_)
        label_test = './dataset/Sensor + Image' + '/' + '{}_label_2_test.npy'.format(sub_)
        img_2_test = np.load(image_test)
        label_2_test = np.load(label_test)
        name_2_test = np.load(name_test)

        # --- Align the test data ---
        redundant_1 = list(set(name_1_test) - set(times_test))
        redundant_2 = list(set(name_2_test) - set(times_test))
        
        ind = np.arange(0, len(img_1_test))

        red_in1 = ind[np.isin(name_1_test, redundant_1)]
        name_1_test = np.delete(name_1_test, red_in1)
        img_1_test = np.delete(img_1_test, red_in1, axis=0)
        label_1_test = np.delete(label_1_test, red_in1)

        red_in2 = ind[np.isin(name_2_test, redundant_2)]
        name_2_test = np.delete(name_2_test, red_in2)
        img_2_test = np.delete(img_2_test, red_in2, axis=0)
        label_2_test = np.delete(label_2_test, red_in2)

        # --- Prepare the final aligned test data ---
        data_test = SUB_test.loc[name_1_test].values

        set_seed()
        X_csv_test, y_csv_test = data_test[:, :-1], data_test[:, -1]
        y_csv_test = np.where(y_csv_test == 20, 0, y_csv_test)
        label_1_test = np.where(label_1_test == 20, 0, label_1_test)
        label_2_test = np.where(label_2_test == 20, 0, label_2_test)

        Y_csv_test = torch.nn.functional.one_hot(torch.from_numpy(y_csv_test).long(), 12).float()
        X_csv_test_scaled = scaled_data(X_csv_test)
        
        X_test_1 = img_1_test
        y_test_1 = label_1_test
        Y_test_1 = torch.nn.functional.one_hot(torch.from_numpy(y_test_1).long(), 12).float()

        X_test_2 = img_2_test
        y_test_2 = label_2_test
        Y_test_2 = torch.nn.functional.one_hot(torch.from_numpy(y_test_2).long(), 12).float()

        X_test_1 = X_test_1.reshape(X_test_1.shape[0], shape1, shape2, 1)
        X_test_2 = X_test_2.reshape(X_test_2.shape[0], shape1, shape2, 1)

        X_test_1_scaled = X_test_1 / 255.0
        X_test_2_scaled = X_test_2 / 255.0

        # --- Store all processed data for the current client ---
        X_train_csv_scaled_splits[clint_index] = X_csv_train_scaled
        X_test_csv_scaled_splits[clint_index] = X_csv_test_scaled
        Y_train_csv_splits[clint_index] = Y_csv_train
        Y_test_csv_splits[clint_index] = Y_csv_test
        X_train_1_scaled_splits[clint_index] = X_train_1_scaled
        X_test_1_scaled_splits[clint_index] = X_test_1_scaled
        Y_train_1_splits[clint_index] = Y_train_1
        Y_test_1_splits[clint_index] = Y_test_1
        X_train_2_scaled_splits[clint_index] = X_train_2_scaled # This line had a bug in the original code
        X_test_2_scaled_splits[clint_index] = X_test_2_scaled
        Y_train_2_splits[clint_index] = Y_train_2
        Y_test_2_splits[clint_index] = Y_test_2
        clint_index += 1
        
    # --- After loop, return all dictionaries ---
    return X_train_csv_scaled_splits,X_test_csv_scaled_splits, Y_train_csv_splits,Y_test_csv_splits,X_train_1_scaled_splits,X_test_1_scaled_splits,Y_train_1_splits,Y_test_1_splits,X_train_2_scaled_splits,X_test_2_scaled_splits,Y_train_2_splits,Y_test_2_splits

## Step 2: Client Selection

We're making great progress. We've handled all the data loading and preparation. Now, we'll add the functions that form the "intelligence" of our federated learning system: client selection.

Instead of blindly averaging updates from every client in every round, these methods evaluate each client's performance and contribution. This allows the server to select the most promising or reliable clients to participate in the global model update, potentially leading to faster convergence and a more robust final model.

We'll add a series of functions, each calculating a specific metric to judge the clients.

### Client Evaluation Metrics
Add all the following functions to your script. Each one calculates a different score based on a client's performance.

1. Relative Loss Reduction (RF_loss)

This measures how much a client's training loss has dropped from the beginning to the end of a local training round, relative to the client with the biggest drop. A higher score means the client is learning effectively.

In [7]:
def calculate_relative_loss_reduction_as_list(client_losses):
    """
    Calculates the relative loss reduction (RF_loss) for each client.
    """
    loss_reduction = {}
    for client_id, losses in client_losses.items():
        if len(losses) < 2:
            raise ValueError(f"Client {client_id} has less than 2 loss values, cannot calculate RF_loss.")
        loss_start = losses[0]
        loss_end = losses[-1]
        loss_reduction[client_id] = loss_start - loss_end

    max_loss_reduction = max(loss_reduction.values())
    if max_loss_reduction == 0:
        return [0.0] * len(loss_reduction)  # If no loss reduction, return 0.0 for all clients

    rf_losses_list = [
        reduction / max_loss_reduction for reduction in loss_reduction.values()
    ]
    return rf_losses_list

2. Relative Training Accuracy (RF_ACC_Train)

This measures a client's local training accuracy relative to the client with the highest accuracy. It's a straightforward measure of performance on local data.

In [8]:
def calculate_relative_train_accuracy(client_acc):
    """
    Calculates the relative training accuracy (RF_Acc_Train) for each client.
    """
    max_acc = max(client_acc.values())
    if max_acc == 0:
        return [0.0] * len(client_acc)  # If no accuracy, return 0.0 for all clients

    rf_accs_train_list = [
        acc / max_acc for acc in client_acc.values()
    ]
    return rf_accs_train_list

3. Global Validation Accuracy (RF_ACC_Global)

This is a more sophisticated metric. It rewards clients for high accuracy on a global test set but penalizes them if their global accuracy is much worse than their local training accuracy (which is a sign of overfitting).

In [9]:
def calculate_global_validation_accuracy(train_acc, global_acc):
    """
    Calculates the global validation accuracy (RF_Acc_Global) based on local training accuracies.
    """
    if set(train_acc.keys()) != set(global_acc.keys()):
        raise ValueError("Client IDs for train and global accuracy do not match.")

    max_global_acc = max(global_acc.values())
    if max_global_acc == 0:
        max_global_acc = 1  # Avoid division by zero

    global_train_diff = {
        client_id: train_acc[client_id] - global_acc[client_id]
        for client_id in train_acc
    }
    max_global_train_diff = max(global_train_diff.values())
    if max_global_train_diff == 0:
        max_global_train_diff = 1  # Avoid division by zero

    rf_acc_global_list = [
        (global_acc[client_id] / max_global_acc) - (global_train_diff[client_id] / max_global_train_diff)
        for client_id in train_acc
    ]
    return rf_acc_global_list

4. Loss Outliers (P_loss)

This function flags clients that are potential negative contributors. If a client's final training loss is significantly higher than the average loss of all clients, it gets a high penalty score. Otherwise, its penalty is zero.

In [10]:
def calculate_loss_outliers(client_losses, lambda_loss=1.5):
    """
    Calculates the loss outlier penalty (P_loss) for each client.
    """
    final_losses = {client_id: losses[-1] for client_id, losses in client_losses.items()}
    loss_values = np.array(list(final_losses.values()))

    mean_loss = np.mean(loss_values)
    std_loss = np.std(loss_values)

    threshold = mean_loss + lambda_loss * std_loss

    max_loss = np.max(loss_values)

    if max_loss == 0:
        return [0.0] * len(loss_values)

    # Identify outliers
    loss_outliers = [
        final_loss / max_loss if final_loss > threshold else 0.0
        for final_loss in loss_values
    ]
    return loss_outliers

5. Performance Bias (P_bias)

This metric calculates the gap between a client's performance on its own validation data versus its performance on the global validation data. A large gap might indicate that the client's local data is not representative of the overall data distribution.

In [11]:
def calculate_performance_bias(val_acc, global_acc):
    """
    Calculates the performance bias penalty (P_bias).
    """
    if set(val_acc.keys()) != set(global_acc.keys()):
        raise ValueError("Client IDs for validation and global accuracy do not match.")

    performance_bias_list = []
    for client_id in val_acc:
        val = val_acc[client_id]
        global_val = global_acc[client_id]
        max_val = max(val, global_val)

        if max_val == 0:
            performance_bias = 0
        else:
            performance_bias = abs(val - global_val) / max_val
        performance_bias_list.append(performance_bias)

    return performance_bias_list

Excellent. Now that we have the functions to score each client, we need the final step: the algorithms that use these scores to select which clients will participate in a given round.

### Client Selection Algorithms
1. Pareto Optimization

This is a powerful technique used when you have multiple, often conflicting, objectives. Instead of combining all metrics into one score, it tries to find a set of clients that represent the best possible trade-offs.

A client is considered "Pareto optimal" if no other client is better than it across all metrics. The algorithm first finds this set of optimal clients.

If there are more optimal clients than needed, it selects a random subset.

If there are fewer, it fills the remaining spots by picking the clients with the best-combined performance score.

In [12]:
def pareto_optimization(
    rf_loss, rf_acc_train, rf_acc_val, rf_acc_global, p_loss, p_bias, client_num
):
    """
    实现 Pareto 优化，筛选节点。

    参数：
    - rf_loss (list): 局部训练损失相对下降幅度。
    - rf_acc_train (list): 局部训练精度。
    - rf_acc_val (list): 局部验证精度。
    - rf_acc_global (list): 全局验证精度。
    - p_loss (list): 损失异常。
    - p_bias (list): 性能偏离。
    - client_num (int): 要选出的节点数。

    返回：
    - selected_clients (list): 选中的 client ID（按输入顺序从 0 开始）。
    """
    # 将输入指标整合为二维数组，便于处理
    # data = np.array([rf_loss, rf_acc_train, rf_acc_val, rf_acc_global, -np.array(p_loss), -np.array(p_bias)]).T

    # 确保所有数组中的元素都转换为 NumPy 数组
    # rf_loss = np.array([x.detach().cpu().numpy() for x in rf_loss])
    rf_loss = np.array(list(rf_loss))
    rf_acc_train = rf_acc_train.detach().cpu().numpy() if isinstance(rf_acc_train, torch.Tensor) else np.array(
        rf_acc_train)
    rf_acc_val = rf_acc_val.detach().cpu().numpy() if isinstance(rf_acc_val, torch.Tensor) else np.array(rf_acc_val)
    rf_acc_global = rf_acc_global.detach().cpu().numpy() if isinstance(rf_acc_global, torch.Tensor) else np.array(
        rf_acc_global)
    p_loss = p_loss.detach().cpu().numpy() if isinstance(p_loss, torch.Tensor) else np.array(p_loss)
    p_bias = p_bias.detach().cpu().numpy() if isinstance(p_bias, torch.Tensor) else np.array(p_bias)
    # rf_acc_train = np.array([x.detach().cpu().numpy() for x in rf_acc_train])
    # rf_acc_val = np.array([x.detach().cpu().numpy() for x in rf_acc_val])
    # rf_acc_global = np.array([x.detach().cpu().numpy() for x in rf_acc_global])
    # p_loss = np.array([x.detach().cpu().numpy() for x in p_loss])
    # p_bias = np.array([x.detach().cpu().numpy() for x in p_bias])

    # 构造 NumPy 数组并转置
    data = np.array([rf_loss, rf_acc_train, rf_acc_val, rf_acc_global, -p_loss, -p_bias]).T

    # Pareto 前沿筛选
    def is_dominated(point, others):
        """判断 point 是否被 others 支配"""
        return any(np.all(other >= point) and np.any(other > point) for other in others)

    pareto_indices = [
        i for i, point in enumerate(data) if not is_dominated(point, np.delete(data, i, axis=0))
    ]
    pareto_clients = pareto_indices

    # 如果前沿节点数多于 client_num，随机选取
    if len(pareto_clients) > client_num:
        return random.sample(pareto_clients, client_num)

    # 如果前沿节点数小于 client_num，基于组合评分补充
    remaining_slots = client_num - len(pareto_clients)
    pareto_scores = [0.4 * rf_loss[i] + 0.6 * rf_acc_global[i] for i in range(len(rf_loss))]
    sorted_indices = np.argsort(pareto_scores)[::-1]  # 按评分从高到低排序

    selected_clients = set(pareto_clients)
    for i in sorted_indices:
        if len(selected_clients) >= client_num:
            break
        if i not in selected_clients:
            selected_clients.add(i)

    return list(selected_clients)

2. Weighted Sum Method (5RF)

This is a more straightforward approach. It calculates a single comprehensive score for each client by taking a weighted sum of all the metrics. Clients with the highest final scores are selected. The weights (0.2, 0.1, 0.3, etc.) determine the importance of each metric.

In [13]:
def get_top_clients_with5RF(rf_loss, rf_acc_train, rf_acc_val, rf_acc_global, p_loss, p_bias, client_num):
    rf_loss = np.array(list(rf_loss))
    rf_acc_train = np.array(rf_acc_train)
    rf_acc_val = np.array(rf_acc_val)
    rf_acc_global = np.array(rf_acc_global)
    p_loss = np.array(p_loss)
    p_bias = np.array(p_bias)

    # Calculate a single weighted score for each client
    scores = (
            0.2 * rf_loss +
            0.1 * rf_acc_train +
            0.2 * rf_acc_val +
            0.3 * rf_acc_global -
            0.1 * p_loss -
            0.1 * p_bias
    )
    origin_scores = scores
    # Get the indices of the clients with the highest scores
    top_client_ids = np.argsort(scores)[::-1][:client_num]  # Sort descending and take the top N
    return top_client_ids.tolist(), origin_scores

## Step 2: The AI's Brain (The Model Definition)
We have the data pipeline and the client selection logic. Now it's time to build the brain of the operation: the neural network model itself.

The model, ModelCSVIMG, is a multi-modal neural network. This means it's designed to accept and process multiple types of data at once. It has three distinct input branches:

One for the numerical sensor (CSV) data.

One for the images from camera 1.

One for the images from camera 2.

The features extracted from each branch are then combined (fused) and passed to a final set of layers that perform the classification. The original code contains a few versions of the architecture; we will use the final, most complex one.

Add the complete model class to your script:

In [14]:
class ModelCSVIMG(nn.Module):
    def __init__(self, num_csv_features, img_shape1, img_shape2):
        super(ModelCSVIMG, self).__init__()

        # --- Branch 1: For processing numerical CSV data ---
        self.csv_fc_1 = nn.Linear(num_csv_features, 2000)
        self.csv_bn_1 = nn.BatchNorm1d(2000)
        self.csv_fc_2 = nn.Linear(2000, 600)
        self.csv_bn_2 = nn.BatchNorm1d(600)
        self.csv_dropout = nn.Dropout(0.2)

        # --- Branch 2: For processing images from Camera 1 (CNN) ---
        self.img1_conv_1 = nn.Conv2d(in_channels=1, out_channels=18, kernel_size=3, stride=1, padding=1)
        self.img1_batch_norm = nn.BatchNorm2d(18)
        self.img1_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Flattened features from the CNN go into a fully connected layer
        self.img1_fc1 = nn.Linear(18 * 16 * 16, 100)
        self.img1_dropout = nn.Dropout(0.2)

        # --- Branch 3: For processing images from Camera 2 (identical to Branch 2) ---
        self.img2_conv = nn.Conv2d(in_channels=1, out_channels=18, kernel_size=3, stride=1, padding=1)
        self.img2_batch_norm = nn.BatchNorm2d(18)
        self.img2_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.img2_fc1 = nn.Linear(18 * 16 * 16, 100)
        self.img2_dropout = nn.Dropout(0.2)

        # --- Fusion and Final Classification Layers ---
        # The input size is 600 (from CSV) + 100 (from Image 1) + 100 (from Image 2) = 800
        self.fc1 = nn.Linear(800, 1200)
        self.dr1 = nn.Dropout(0.2)
        # A residual connection is used here: input to fc2 is the original 800 + output of fc1 (1200) = 2000
        self.fc2 = nn.Linear(2000, 12) # 12 output classes

    def forward(self, x_csv, x_img1, x_img2):
        # --- Process CSV data ---
        x_csv = F.relu(self.csv_bn_1(self.csv_fc_1(x_csv)))
        x_csv = F.relu(self.csv_bn_2(self.csv_fc_2(x_csv)))
        x_csv = self.csv_dropout(x_csv)

        # --- Process Image 1 data ---
        # Reshape image from (batch, height, width, channels) to (batch, channels, height, width)
        x_img1 = x_img1.permute(0, 3, 1, 2)
        x_img1 = F.relu(self.img1_conv_1(x_img1))
        x_img1 = self.img1_batch_norm(x_img1)
        x_img1 = self.img1_pool(x_img1)
        x_img1 = x_img1.contiguous().view(x_img1.size(0), -1) # Flatten
        x_img1 = F.relu(self.img1_fc1(x_img1))
        x_img1 = self.img1_dropout(x_img1)

        # --- Process Image 2 data ---
        x_img2 = x_img2.permute(0, 3, 1, 2)
        x_img2 = F.relu(self.img2_conv(x_img2))
        x_img2 = self.img2_batch_norm(x_img2)
        x_img2 = self.img2_pool(x_img2)
        x_img2 = x_img2.contiguous().view(x_img2.size(0), -1) # Flatten
        x_img2 = F.relu(self.img2_fc1(x_img2))
        x_img2 = self.img2_dropout(x_img2)

        # --- Fusion ---
        x = torch.cat((x_csv, x_img1, x_img2), dim=1)
        residual = x # Keep a copy for the residual connection
        
        # --- Final layers ---
        x = F.relu(self.fc1(x))
        x = self.dr1(x)
        # Concatenate the residual connection
        x = torch.cat((residual, x), dim=1)
        # Final output with softmax for classification
        x = F.softmax(self.fc2(x), dim=1)

        return x

## Step 3: The Teacher (The Server Class)
Alright, we're on the home stretch. We have the data, the selection logic, and the model. Now we need to create the actors for our simulation: the Server and the Client. These two classes will control the entire federated learning process.

1. The Server Class

The Server is the central coordinator. Its job is to:

Hold the main global model.

Send the global model to the clients.

Receive updates from the selected clients.

Aggregate these updates to improve the global model.

Evaluate the global model's performance on a held-out test set.

Here is the code for the Server.

In [15]:
# Server
class Server(object):
    def __init__(self, model, epoch_size, eval_dataset, num_clients):
        self.global_model = model
        self.epoch_size = epoch_size
        self.num_clients = num_clients
        self.serverTestDataSet = CustomDatasetRes(eval_dataset[0],eval_dataset[1],eval_dataset[2],eval_dataset[3])
        self.eval_loader = torch.utils.data.DataLoader(self.serverTestDataSet, batch_size=epoch_size)

    def model_aggregate(self, weight_accumulator):
        # Averages the weights from the selected clients and updates the global model
        for name, data in self.global_model.state_dict().items():
            update_per_layer = weight_accumulator[name] * (1/self.num_clients)   # average
            if data.type() != update_per_layer.type():
                data.add_(update_per_layer.to(torch.int64))
            else:
                data.add_(update_per_layer)

    def model_eval(self):
        # Evaluates the global model on the server's test data
        self.global_model.eval()
        total_loss = 0.0
        correct = 0
        dataset_size = 0
        with torch.no_grad(): # Disable gradient calculation for evaluation
            for batch_id, batch in enumerate(self.eval_loader):
                data1, data2, data3, target = batch
                dataset_size += data1.size()[0]

                data1 = data1.to(device).float()
                data2 = data2.to(device).float()
                data3 = data3.to(device).float()
                target = target.to(device).float()
                
                output = self.global_model(data1,data2,data3)
                total_loss += nn.functional.cross_entropy(output, target, reduction='sum').item()

                pred = output.detach().max(1)[1]
                correct += pred.eq(target.detach().max(1)[1].view_as(pred)).cpu().sum().item()

        acc = 100.0 * (float(correct) / float(dataset_size))
        loss = total_loss / dataset_size
        return acc, loss

2. The Client Class and Helper Functions

The Client represents an individual participant. Its job is to:

Receive the global model from the server.

Train this model on its own local data for a few epochs.

Calculate the change (the diff) between the original model and its newly trained model.

Send this diff back to the server.

The client's training process is handled by two helper functions: train_one_epoch and validate.

Add the Client class and its two helper functions to your script.

In [16]:
# Client
class Client(object):
    def __init__(self, epoch_size, local_epoch_per_round, train_dataset,val_dataset, id = -1):
        self.epoch_size = epoch_size
        self.local_epoch_per_round = local_epoch_per_round
        self.client_id = id
        self.train_dataset = CustomDatasetRes(train_dataset[0],train_dataset[1],train_dataset[2],train_dataset[3])
        self.train_loader = torch.utils.data.DataLoader(self.train_dataset, batch_size=epoch_size,shuffle=True)
        self.eval_dataset = CustomDatasetRes(val_dataset[0], val_dataset[1], val_dataset[2], val_dataset[3])
        self.eval_loader = torch.utils.data.DataLoader(self.eval_dataset, batch_size=epoch_size,shuffle=False)

    def local_train(self, global_model):
        # Create a local copy of the global model
        model = ModelCSVIMG(self.train_dataset.features1.shape[1], 32, 32)
        model = model.to(device)
        model.load_state_dict(global_model.state_dict())

        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        
        losses = []
        min_loss, max_loss = float('inf'), float('-inf')

        for epoch in range(self.local_epoch_per_round):
            # Train for one epoch
            train_loss, train_acc = train_one_epoch(model, self.train_loader, criterion, optimizer)
            
            # Track min/max loss and store all epoch losses for evaluation
            if train_loss > max_loss: max_loss = train_loss
            if train_loss < min_loss: min_loss = train_loss
            losses.append(train_loss)

        # Validate the model on the local validation set after training
        val_loss, val_acc = validate(model, self.eval_loader, criterion)
        print(f"Client {self.client_id} - Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%")

        # Calculate the difference between the updated local model and the original global model
        diff = dict()
        for name, data in model.state_dict().items():
            diff[name] = (data - global_model.state_dict()[name])
            
        return model, diff, val_acc, val_loss, min_loss, max_loss, losses, train_acc


def train_one_epoch(model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_id, batch in enumerate(train_loader):
        data1, data2, data3, target = batch
        data1 = data1.to(device).float()
        data2 = data2.to(device).float()
        data3 = data3.to(device).float()
        target = target.to(device).float()

        optimizer.zero_grad()
        output = model(data1, data2, data3)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * data1.size(0)
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target.max(1)[1]).sum().item()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = 100.0 * correct / total
    return epoch_loss, epoch_acc

def validate(model, val_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_id, batch in enumerate(val_loader):
            data1, data2, data3, target = batch
            data1 = data1.to(device).float()
            data2 = data2.to(device).float()
            data3 = data3.to(device).float()
            target = target.to(device).float()

            output = model(data1, data2, data3)
            loss = criterion(output, target)

            running_loss += loss.item() * data1.size(0)
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target.max(1)[1]).sum().item()

    epoch_loss = running_loss / len(val_loader.dataset)
    epoch_acc = 100.0 * correct / total
    return epoch_loss, epoch_acc

We are almost there! We've built all the major components. Before we assemble everything in the main training loop, we need to add the last few helper functions.

These functions are primarily used for a simpler, baseline client selection strategy (referred to as '4RF' in the code) that calculates a single score for each client and picks the best ones.

Add these final utility functions to your script.

1. Normalize

A standard function to scale any number to a range between 0 and 1, given a minimum and maximum value. This is useful for combining metrics that have different scales.

In [17]:
def normalize(value, min_value, max_value):
    # Avoid division by zero if min and max are the same
    if (max_value - min_value) == 0:
        return 0
    return (value - min_value) / (max_value - min_value)

2. Evaluate Model Score

This function calculates a simple, combined score for a client. It's a weighted average of their performance on their local training set and a global validation set.

In [18]:
def evaluate_model(acc, loss, min_loss, max_loss, oneclient_test_acc, oneclient_test_loss,alpha=0.8, beta=0.8):
    normalized_loss = normalize(loss, min_loss, max_loss)
    # Score based on local training performance
    train_score = alpha * acc + (1 - alpha) * (1 - normalized_loss) # Use (1 - loss) so higher is better

    # Score based on global validation performance
    val_score = beta * oneclient_test_acc + (1 - beta) * (1 - oneclient_test_loss)

    # Final combined score
    combined_score = (train_score + val_score) / 2
    return combined_score

3. Get Top Clients

A straightforward function that takes a dictionary of clients and their scores, then returns a list of the top num clients with the highest scores.

In [19]:
def get_top_clients(client_dict, num):
    # Sort the clients by their score (the dictionary value) in descending order
    sorted_clients = sorted(client_dict.items(), key=lambda item: item[1], reverse=True)
    # Extract just the IDs (the dictionary key) of the top clients
    top_clients = [client[0] for client in sorted_clients[:num]]
    return top_clients

4. Dynamic Threshold Selection

This is another, more advanced selection method included in the script. It selects clients whose scores are above a dynamic threshold (calculated from the mean and standard deviation of all scores). While not used in the final configuration, we include it for completeness.

In [20]:
def select_nodes_with_dynamic_threshold(node_scores, max_nodes, std_multiplier=1.0):
    """
    Selects nodes using a dynamic threshold based on score distribution.
    """
    if not node_scores:
        return []
        
    scores = np.array(list(node_scores.values()))
    
    # Calculate the dynamic threshold
    mean_score = np.mean(scores)
    std_dev = np.std(scores)
    dynamic_threshold = mean_score + std_multiplier * std_dev

    # Select nodes above the threshold
    selected_nodes = [
        node_id for node_id, score in node_scores.items() if score >= dynamic_threshold
    ]

    # If too many nodes were selected, keep only the best ones
    if len(selected_nodes) > max_nodes:
        selected_nodes = sorted(
            selected_nodes, key=lambda node_id: node_scores[node_id], reverse=True
        )[:max_nodes]

    # If not enough nodes were selected, add the next best ones to meet the quota
    if len(selected_nodes) < max_nodes:
        remaining_nodes = [
            node_id for node_id in node_scores if node_id not in selected_nodes
        ]
        remaining_nodes = sorted(
            remaining_nodes, key=lambda node_id: node_scores[node_id], reverse=True
        )
        selected_nodes += remaining_nodes[: max_nodes - len(selected_nodes)]

    return selected_nodes

## Step 4: Model Trainer 

Here we go. This is the big one. We'll now write the trainValModelCSVIMG function. This function is the conductor of our orchestra—it brings together the data, the model, the server, and the clients to run the entire federated learning simulation from start to finish.

Because it's so long and important, we'll build it in three parts.

### Part 1: Initialization and Starting the Training Loop

First, we'll define the function and set everything up. This includes:

Creating the global model, the Server, and all the Client objects.

Initializing a series of dictionaries to log every possible metric (loss, accuracy, selection scores, etc.) for every client and every round. This is crucial for analyzing the experiment later.

Starting the main training loop, which iterates through the communication rounds.

Inside the loop, we'll begin the first phase of a round: every client trains locally on the current global model.

### Part 2: Client Selection, Aggregation, and Global Evaluation
In this part of the trainValModelCSVIMG function, the server performs the following steps:

Evaluate: It uses all the metrics gathered from the clients to calculate the advanced performance scores (RF_loss, P_bias, etc.).

Select: Based on the chosen selection method (svmethod), it picks the top-performing clients for this round.

Aggregate: It averages the model updates (diffs) from only the selected clients to create a new, improved global model.

Evaluate Globally: It tests the new global model's performance on the entire held-out test set.

Save Best Model: If the new global model is the best one seen so far, its state is saved to a file.

### Part 3: Final Test and Saving Results
Now that the training is finished, we need to do two last things:

Load the best model that we saved during training and run a final, definitive test on it. This gives us the final performance numbers for our experiment.

Save all the logs we've been collecting into a CSV file. This is essential for creating plots and analyzing the training process, client behavior, and the effectiveness of the selection strategy.

In [21]:
def trainValModelCSVIMG(model_name, svmethod, total_client,num_clients,epoch,max_acc,epoch_size,local_epoch_per_round,round_early_stop,
                        X_train_csv_scaled_splits, X_test_csv_scaled_splits,
                        X_train_1_scaled_splits, X_test_1_scaled_splits,
                        X_train_2_scaled_splits, X_test_2_scaled_splits,
                        Y_train_csv_splits, Y_test_csv_splits):
    # --- 1. Initialization ---
    # Instantiate the global model and move it to the GPU if available
    model_MLP = ModelCSVIMG(X_train_csv_scaled_splits[0].shape[1], 32, 32)
    model_MLP = model_MLP.to(device)

    # The last client's test data is reserved for the server's global evaluation
    server = Server(model_MLP, epoch_size, [X_test_csv_scaled_splits[total_client-1], X_test_1_scaled_splits[total_client-1], X_test_2_scaled_splits[total_client-1], Y_test_csv_splits[total_client-1]], num_clients)
    
    # Create a list of all clients
    clients = []
    for client_index in range(total_client):
        clients.append(Client(epoch_size=epoch_size, local_epoch_per_round=local_epoch_per_round,
                              train_dataset=[X_train_csv_scaled_splits[client_index], X_train_1_scaled_splits[client_index], X_train_2_scaled_splits[client_index],
                                             Y_train_csv_splits[client_index]],
                              val_dataset=[X_test_csv_scaled_splits[client_index], X_test_1_scaled_splits[client_index], X_test_2_scaled_splits[client_index],
                                           Y_test_csv_splits[client_index]], id=client_index))

    # --- Dictionaries for Logging ---
    clients_scoresDict = {}
    perEpoch_clients_losses = {}
    perEpoch_clients_train_acc = {}
    perEpoch_clients_local_test_acc = {}
    perEpoch_clients_global_test_acc = {}
    clients_train_acc = {}
    clients_train_loss = {}
    clients_test_acc = {}
    clients_test_loss = {}
    clients_rf_relative_loss_reduction = {}
    clients_rf_acc_train = {}
    clients_rf_global_validation_accuracy = {}
    clients_rf_loss_outliers = {}
    clients_rf_performance_bias = {}
    clients_epoch_selected = {}

    for i in range(total_client + 1):  # +1 for the server/global model
        clients_train_acc[i] = []
        clients_train_loss[i] = []
        clients_test_acc[i] = []
        clients_test_loss[i] = []
        clients_scoresDict[i] = []
        clients_rf_relative_loss_reduction[i] = []
        clients_rf_acc_train[i] = []
        clients_rf_global_validation_accuracy[i] = []
        clients_rf_loss_outliers[i] = []
        clients_rf_performance_bias[i] = []
        clients_epoch_selected[i] = []

    epoch_count = 0
    # --- 2. Main Federated Learning Loop ---
    for e in range(epoch):
        print(f"--- Round {e+1}/{epoch} ---")
        if epoch_count >= round_early_stop:
            print("Early stopping triggered.")
            break
            
        diff_client = {}
        weight_accumulator = {}
        for name, params in server.global_model.state_dict().items():
            weight_accumulator[name] = torch.zeros_like(params)

        # --- Phase 1: All clients perform local training ---
        for client_index in range(total_client):
            round_client_model, diff, test_acc_client, loss_client, min_loss, max_loss, losses, train_acc = clients[client_index].local_train(server.global_model)
            
            # Store results for this client
            perEpoch_clients_losses[client_index] = losses
            perEpoch_clients_train_acc[client_index] = train_acc
            perEpoch_clients_local_test_acc[client_index] = test_acc_client
            diff_client[client_index] = diff
            
            # Evaluate this client's trained model on the entire global test set
            correct = 0
            dataset_size = 0
            with torch.no_grad():
                for test_data_index in range(total_client): # Loop through all test splits
                    test_server_loader = torch.utils.data.DataLoader(
                        CustomDatasetRes(X_test_csv_scaled_splits[test_data_index], X_test_1_scaled_splits[test_data_index],
                                      X_test_2_scaled_splits[test_data_index], Y_test_csv_splits[test_data_index]),
                        batch_size=epoch_size)
                    
                    round_client_model.eval()
                    for batch_id, batch in enumerate(test_server_loader):
                        data1, data2, data3, target = batch
                        dataset_size += data1.size()[0]
                        data1, data2, data3, target = data1.to(device).float(), data2.to(device).float(), data3.to(device).float(), target.to(device).float()
                        
                        output = round_client_model(data1, data2, data3)
                        pred = output.detach().max(1)[1]
                        correct += pred.eq(target.detach().max(1)[1].view_as(pred)).cpu().sum().item()

            oneclient_global_test_acc = 100.0 * (correct / dataset_size)
            perEpoch_clients_global_test_acc[client_index] = oneclient_global_test_acc
            
            # Log the local and global test accuracies for this round
            clients_train_acc[client_index].append(test_acc_client)
            clients_train_loss[client_index].append(loss_client)
            clients_test_acc[client_index].append(oneclient_global_test_acc)
            clients_epoch_selected[client_index].append(0) # Mark as not selected (yet)

        # --- Phase 2: Server evaluates, selects, and aggregates ---
        # Calculate all the advanced performance metrics for each client
        rf_relative_loss_reduction = calculate_relative_loss_reduction_as_list(perEpoch_clients_losses)
        rf_acc_train = calculate_relative_train_accuracy(perEpoch_clients_train_acc)
        rf_global_validation_accuracy = calculate_global_validation_accuracy(perEpoch_clients_train_acc, perEpoch_clients_global_test_acc)
        rf_loss_outliers = calculate_loss_outliers(perEpoch_clients_losses)
        rf_performance_bias = calculate_performance_bias(perEpoch_clients_local_test_acc, perEpoch_clients_global_test_acc)
        
        # Log these calculated metrics
        for client_index in range(total_client):
            clients_rf_relative_loss_reduction[client_index].append(rf_relative_loss_reduction[client_index])
            clients_rf_acc_train[client_index].append(rf_acc_train[client_index])
            clients_rf_global_validation_accuracy[client_index].append(rf_global_validation_accuracy[client_index])
            clients_rf_loss_outliers[client_index].append(rf_loss_outliers[client_index])
            clients_rf_performance_bias[client_index].append(rf_performance_bias[client_index])

        # --- Select clients based on the specified method ---
        candidates = []
        if svmethod == '5RF':
            # Note: rf_acc_val is not defined in the code, so we'll pass an empty list for now.
            rf_acc_test = [acc[-1] for acc in clients_test_acc.values() if acc]
            candidates, scores = get_top_clients_with5RF(rf_relative_loss_reduction, rf_acc_train, rf_acc_test,
                                                         rf_global_validation_accuracy, rf_loss_outliers, rf_performance_bias,
                                                         num_clients)
            for index in range(len(scores)):
                clients_scoresDict[index].append(scores[index])
        elif svmethod == 'pareto':
            rf_acc_test = [acc[-1] for acc in clients_test_acc.values() if acc]
            candidates = pareto_optimization(rf_relative_loss_reduction, rf_acc_train, rf_acc_test,
                                              rf_global_validation_accuracy, rf_loss_outliers, rf_performance_bias,
                                              num_clients)
        elif svmethod == 'random':
            candidates = np.random.choice(total_client, num_clients, replace=False)

        print(f"Selected clients for aggregation: {candidates}")

        # Mark which clients were selected in the log
        for selected_client_index in candidates:
            clients_epoch_selected[selected_client_index][-1] = 1
        
        # --- Aggregate the updates from selected clients ---
        for slected_client_index in candidates:
            for name, params in server.global_model.state_dict().items():
                weight_accumulator[name].add_(diff_client[slected_client_index][name])
        
        server.model_aggregate(weight_accumulator)

        # --- Phase 3: Evaluate the new global model ---
        acc, loss = server.model_eval()
        
        # Log global model performance
        clients_test_acc[total_client].append(acc)
        clients_test_loss[total_client].append(loss)
        
        print(f"Round {e+1} Global Model - Accuracy: {acc:.2f}%, Loss: {loss:.4f}\n")

        # --- Save the best model and check for early stopping ---
        epoch_count += 1
        if acc > max_acc:
            max_acc = acc
            print("New best model found! Saving model...")
            torch.save(server.global_model.state_dict(),
                       f"./acc_lossFiles/{model_name}_totalClient_{total_client}_NumClient_{num_clients}_epoch_{epoch}_svmethod_{svmethod}.pth")
            epoch_count = 0 # Reset early stopping counter
        
    # --- After the training loop, perform a final evaluation on the best model ---
    print("\n--- Final Evaluation on Best Model ---")
    model = model_MLP
    model.load_state_dict(torch.load(
        f"./acc_lossFiles/{model_name}_totalClient_{total_client}_NumClient_{num_clients}_epoch_{epoch}_svmethod_{svmethod}.pth"))
    model = model.to(device)
    model.eval()
    
    y_test, y_predict = [], []
    total_loss = 0.0
    correct = 0
    dataset_size = 0

    with torch.no_grad():
        for test_data_index in range(total_client):
            test_server_loader = torch.utils.data.DataLoader(
                CustomDatasetRes(X_test_csv_scaled_splits[test_data_index], X_test_1_scaled_splits[test_data_index],
                              X_test_2_scaled_splits[test_data_index], Y_test_csv_splits[test_data_index]),
                batch_size=epoch_size)
            for batch_id, batch in enumerate(test_server_loader):
                data1, data2, data3, target = batch
                dataset_size += data1.size()[0]
                data1, data2, data3, target = data1.to(device).float(), data2.to(device).float(), data3.to(device).float(), target.to(device).float()

                output = model(data1, data2, data3)
                total_loss += nn.functional.cross_entropy(output, target, reduction='sum').item()
                
                pred = output.detach().max(1)[1]
                correct += pred.eq(target.detach().max(1)[1].view_as(pred)).cpu().sum().item()

    acc = 100.0 * (correct / dataset_size)
    loss = total_loss / dataset_size

    print(f'Final Best Model Test Accuracy: {acc:.2f}%')
    print(f'Final Best Model Test Loss: {loss:.4f}')
    print(f'Max accuracy achieved during training: {max_acc:.2f}%')

    # --- Save all logged data to a CSV file ---
    """csv_file_name = f"./acc_lossFiles/{model_name}_totalClient_{total_client}_NumClient_{num_clients}_epoch_{epoch}_svmethod_{svmethod}.csv"
    try:
        # 保存到 CSV 文件
        with open(csv_file_name, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['server_client_index',server_client_index])
            writer.writerow(
                ['client', 'Epoch', 'train_Loss', 'train_Accuracy', 'test_Loss', 'test_Accuracy','rf_loss', 'rf_acc_train', 'rf_acc_val', 'rf_acc_global', 'p_loss', 'p_bias', 'selected'])
            for i in range(total_client+1):
                print('i:',i)
                # 添加列名
                train_losses = clients_train_loss[i]
                train_accs = clients_train_acc[i]
                test_loss = clients_test_loss[i]
                test_acc = clients_test_acc[i]
                rf_loss = clients_rf_relative_loss_reduction[i]
                rf_acc_train = clients_rf_acc_train[i]
                rf_acc_global = clients_rf_global_validation_accuracy[i]
                p_loss = clients_rf_loss_outliers[i]
                p_bias = clients_rf_performance_bias[i]
                selecteds = clients_epoch_selected[i]
                if i == total_client:
                    for j in range(len(train_losses)):
                        writer.writerow([i, j + 1, train_losses[j], train_accs[j], test_loss[j], test_acc[j], 0,0,0,0,0,0,0])
                else:
                    for j in range(len(train_losses)):
                        writer.writerow([i, j + 1, train_losses[j], train_accs[j], test_loss[j], test_acc[j],rf_loss[j],rf_acc_train[j], rf_acc_global[j],p_loss[j],p_bias[j],selecteds[j]])
        print(f"Results saved to {csv_file_name}")
    except Exception as e:
        print(f"Error occurred while saving results: {e}")
        continue"""

We've arrived at the final step! We have all the building blocks in place. The only thing left is to set our experimental parameters and create the main execution block that calls our functions and runs the simulation.

## Final Step: The main Function and Execution Block
This final piece of code does the following:

Sets Hyperparameters: Defines all the key variables for the experiment, like the number of clients, epochs, learning rate, etc. It also defines the different scenarios we want to test (e.g., different client selection methods, different data corruption scenarios).

Defines a main() function: This function orchestrates the experiment. It loads the client data, then loops through each experimental scenario. For scenarios involving "model loss," it intentionally corrupts the data for some clients (e.g., replacing their sensor data with random noise) to simulate system failures or unreliable participants.

Calls trainValModelCSVIMG: For each scenario, it calls our main training function to run a full federated learning simulation.

Executes main(): The standard if __name__ == "__main__": line ensures that the main function is called when you run the script.

In [ ]:
# --- Define Experimental Scenarios and Hyperparameters ---
model_names = {'tc1c2ResModelV3DataV3Adam', 
               'tc1c2ResModelV3DataV3AdamWithSCVLost', 
               'tc1c2ResModelV3DataV3AdamWithImgLost'}

model_names = {'tc1c2ResModelV3DataV3AdamWithImgLost'}
svmethods = {'pareto', '5RF', 'random'}
svmethods = {'pareto'}  # For testing purposes, you can change this to 'pareto' or 'random'
# --- Hyperparameters ---
max_acc = 1  # Threshold of accuracy for saving the best model
epoch = 2  # Total number of communication rounds
epoch_size = 64  # Batch size
total_client = 12  # Total number of clients in the pool
num_clients = 6  # Number of clients selected per round
local_epoch_per_round = 3 # Number of local training epochs for each client per round
round_early_stop = 10 # Number of rounds without improvement before stopping
server_client_index = random.randint(0, total_client - 1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def main():
    # Load the data pre-split for each of the 12 clients
    X_train_csv_scaled_splits, X_test_csv_scaled_splits, \
    Y_train_csv_splits, Y_test_csv_splits, \
    X_train_1_scaled_splits, X_test_1_scaled_splits, \
    Y_train_1_splits, Y_test_1_splits, \
    X_train_2_scaled_splits, X_test_2_scaled_splits, \
    Y_train_2_splits, Y_test_2_splits = loadClientsData()

    # Loop through each experimental scenario
    for model_name in model_names:
        # --- Simulate Data Corruption Scenarios ---
        if model_name == 'tc1c2ResModelV3DataV3AdamWithSCVLost':
            print("\n--- Running Scenario: Corrupted CSV Data ---")
            # Replace CSV data with random noise for clients 6 through 11
            for index in [6, 7, 8, 9, 10, 11]:
                shape_train = X_train_csv_scaled_splits[index].shape
                X_train_csv_scaled_splits[index] = np.random.rand(*shape_train)
                shape_test = X_test_csv_scaled_splits[index].shape
                X_test_csv_scaled_splits[index] = np.random.rand(*shape_test)
        elif model_name == 'tc1c2ResModelV3DataV3AdamWithImgLost':
            print("\n--- Running Scenario: Corrupted Image Data ---")
            # Replace Image data with random noise for clients 6 through 11
            for index in [6, 7, 8, 9, 10, 11]:
                shape_train1 = X_train_1_scaled_splits[index].shape
                X_train_1_scaled_splits[index] = np.random.rand(*shape_train1)
                shape_test1 = X_test_1_scaled_splits[index].shape
                X_test_1_scaled_splits[index] = np.random.rand(*shape_test1)
                shape_train2 = X_train_2_scaled_splits[index].shape
                X_train_2_scaled_splits[index] = np.random.rand(*shape_train2)
                shape_test2 = X_test_2_scaled_splits[index].shape
                X_test_2_scaled_splits[index] = np.random.rand(*shape_test2)

        # Loop through each client selection method
        for svmethod in svmethods:
            print(f"\n===== STARTING NEW EXPERIMENT: Model={model_name}, Selection={svmethod} =====")
            trainValModelCSVIMG(model_name, svmethod, total_client, num_clients, epoch, max_acc, epoch_size, local_epoch_per_round, round_early_stop,
                                X_train_csv_scaled_splits, X_test_csv_scaled_splits,
                                X_train_1_scaled_splits, X_test_1_scaled_splits,
                                X_train_2_scaled_splits, X_test_2_scaled_splits,
                                Y_train_csv_splits, Y_test_csv_splits)

# This makes the script runnable
if __name__ == "__main__":
    main()


--- Running Scenario: Corrupted Image Data ---

===== STARTING NEW EXPERIMENT: Model=tc1c2ResModelV3DataV3AdamWithImgLost, Selection=pareto =====
--- Round 1/2 ---
Client 0 - Train Acc: 95.61%, Val Acc: 96.05%
Client 1 - Train Acc: 93.31%, Val Acc: 94.28%
Client 2 - Train Acc: 96.49%, Val Acc: 95.86%


KeyboardInterrupt: 